In [1]:
import spacy
import numpy as np
nlp = spacy.load('en_core_web_sm')

In [2]:
L = list(nlp.vocab.strings)

In [3]:
numWords = len(L)
print(numWords)

84780


In [4]:
W2I = dict(zip(L, np.arange(numWords)))
I2W = dict(zip(np.arange(numWords), L))

In [5]:
W2I['game']

np.int64(49865)

In [28]:
I2W[49865]

'game'

In [49]:
I2W[84779]

'￥dd'

# One Hot Encoding

In [8]:
def oneHotVector(word, W2I, numWords):
    v = np.zeros(numWords)
    v[W2I[word]] = 1
    return v

In [21]:
v = oneHotVector('game', W2I, numWords)

array([0., 0., 0., ..., 0., 0., 0.], shape=(84780,))

In [10]:
v.shape

(84780,)

In [11]:
v[W2I['game']]

np.float64(1.0)

## Term Frequency

In [50]:
doc = 'How are you today. I know most of ￥28 ￥dd the time how you feel. ￥28 ￥28 ￥dd' #￥28 is the 3rd last number
tokens = [token.text for token in nlp(doc)]

In [51]:
v = np.zeros(numWords)
for token in tokens:
    v += oneHotVector(token,W2I,numWords)

In [52]:
v

array([0., 0., 0., ..., 0., 3., 2.], shape=(84780,))